In [1]:
import pandas as pd
from libraries.utility import Utility as mutil
from libraries.db_base import db_base
import os
import time

from libraries.utility import Utility as mutil
from libraries.db_base import db_base
from libraries.db_ins_fake_data import fake_data_to_db
from libraries.custom_excel_output import custom_excel_output
import pandas as pd
import pandasql as psql
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import warnings
import time
import os
import pandasql as psql
from IPython.core.display import Markdown as md
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML, Markdown, Latex
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '.end_space { min-height:0px !important; } '
        '.prompt {width: 0px; min-width: 0px; visibility: collapse } '
        '.parent{'
        '    display: grid;'
        '    grid-template-columns: 1fr 1fr;'
        '    column-gap: 5px;'
        '}    '
    '</style>'
))


from pyspark.sql import SparkSession



In [2]:

u = mutil()

bq_credential_file = os.path.join(u.get_this_dir(),"dbs","servers","bigquery","bigquery1","tokens","brave-sonar-367918-74b1d5b6db90.json")

    #Location of database drivers
    #postgres
    #/usr/share/java/postgresql.jar
    #mysql
    #/usr/share/java/mysql-connector-j-8.2.0.jar
    #mssql
    #/usr/share/java/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre11.jar

def create_spark_sesssion():
    # Create SparkSession
    spark = SparkSession.builder \
        .appName("MasterSession") \
        .config("spark.jars", "/usr/share/java/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre11.jar") \
        .config("spark.jars", "/usr/share/java/postgresql.jar") \
        .config("spark.jars", "/usr/share/java/mysql-connector-j-8.2.0.jar") \
        .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.21.1') \
        .config('spark.jars', 'https://storage.googleapis.com/spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
        .config("credentialsFile", bq_credential_file) \
        .config("parentProject", "brave-sonar-367918") \
        .getOrCreate()
    return spark

def test_sql_server_connection(query,spark):
    

    # Start the timer
    start_time = time.time()
    


    # Database connection properties
    database_url = "jdbc:sqlserver://mssql1:1433;databaseName=products"
    database_properties = {
        "user": "sa",
        "password": "Python2028",
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
        "encrypt": "true",
        "trustServerCertificate": "true"  # Add this line        
    }

    # Read data from MSSQL
    #df = spark.read.jdbc(url=database_url, table="products", properties=database_properties)

    # Read data from MSSQL
    df = spark.read.jdbc(url=database_url, table=query, properties=database_properties)

    # Show the data
    #df.show()
    
    # Stop the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print(f"Time elapsed: {minutes} minutes and {seconds} seconds")
    return df


def test_postgress_sql(query,spark):
  

    # Start the timer
    start_time = time.time()
    

    # Database connection properties
    database_url = "jdbc:postgresql://postsql1:5432/finance"
    database_properties = {
        "user": "postgres",
        "password": "Python2028",
        "driver": "org.postgresql.Driver",
        "encrypt": "true",
        "trustServerCertificate": "true"  # Add this line        
    }
    
    # Read data from MSSQL
    df = spark.read.jdbc(url=database_url, table=query, properties=database_properties)

    # Show the data
    #df.show()
    
    # Stop the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print(f"Time elapsed: {minutes} minutes and {seconds} seconds")
    return df

def test_mysql_server(query,spark):
    
    # Start the timer
    start_time = time.time()
    

    # Database connection properties
    database_url = "jdbc:mysql://mysql1:3306/customers"
    database_properties = {
        "user": "root",
        "password": "Python2028",
        "driver": "com.mysql.cj.jdbc.Driver",
        "encrypt": "true",
        "trustServerCertificate": "true"  # Add this line        
    }
    
    # Read data from MSSQL
    df = spark.read.jdbc(url=database_url, table=query, properties=database_properties)
    
    # Stop the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print(f"Time elapsed: {minutes} minutes and {seconds} seconds")
    
    return df
    


In [3]:

u = mutil()
# Start the timer
start_time = time.time()

# Initialize a SparkSession
spark = create_spark_sesssion()


dbprod = db_base("products")
get_product_info_sql=dbprod.get_sql_query_from_query_key("get_product_info")

query= "(" + get_product_info_sql + ") AS query_table"

prod_info=test_sql_server_connection(query=query,spark=spark)

dbfin = db_base("finance",svr_type='postsql')

get_account_activity = dbfin.get_sql_query_from_query_key("get_account_activity")

query= "(" + get_account_activity + ") AS query_table"

fin_account_activity=test_postgress_sql(query,spark)

dbcust = db_base("customers",svr_type='mysql')

query = dbcust.get_sql_query_from_query_key("get_customer_product_history1")

query = "(" + query + ") AS query_table"

cust_products = test_mysql_server(query,spark)


/spark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2ded9387-0727-4697-b63c-3d4d8e992921;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.21.1 in central
downloading https://repo1.maven.org/maven2/com/google/cloud/spark/spark-bigquery-with-dependencies_2.12/0.21.1/spark-bigquery-with-dependencies_2.12-0.21.1.jar ...
	[SUCCESSFUL ] com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.21.1!spark-bigquery-with-dependencies_2.12.jar (4954ms)
:: resolution report :: resolve 608ms :: artifacts dl 4956ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.21.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifa

('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 18 for SQL Server' : file not found (0) (SQLDriverConnect)")
Database : products, Connection Good: False
Time elapsed: 0 minutes and 1 seconds
connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

Database : finance, Connection Good: False
Time elapsed: 0 minutes and 0 seconds
2003 (HY000): Can't connect to MySQL server on '127.0.0.1:3306' (111)
Database : customers, Connection Good: False
Time elapsed: 0 minutes and 0 seconds


In [4]:

# Register DataFrames as temporary views
cust_products.createOrReplaceTempView("cust_products")
prod_info.createOrReplaceTempView("prod_info")
fin_account_activity.createOrReplaceTempView("fin_account_activity")


# SQL query
sql_query = '''
SELECT
    DISTINCT
    cp.id AS customer_id,
    cp.f_name,
    cp.l_name,
    p.product_name,
    p.product_type,
    fa.amt_usd,
    fa.post_date,
    fa.channel_desc,
    fa.partner_desc,
    fa.location_name,
    fa.account_name,
    fa.account_type
FROM cust_products cp
JOIN prod_info p ON p.id = cp.product_id
JOIN fin_account_activity fa ON fa.customer_id = cp.id AND fa.product_id = cp.product_id
'''

# Execute SQL query
result_rdd = spark.sql(sql_query)
result_df = result_rdd.toPandas()
#result_df_bq = result_rdd.toDF()


In [5]:

#Write the stuff to my bigquery database.
result_rdd.write \
  .format("bigquery") \
  .option("project", "brave-sonar-367918") \
  .option("table", "site_traffic.financial_data") \
  .option("writeMethod", "direct") \
  .mode("append") \
  .save()



# Stop the SparkSession when done
spark.stop()


# Stop the timer and calculate elapsed time
end_time = time.time()
elapsed_time = end_time - start_time

# Convert elapsed time to minutes and seconds
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"Overall Time: {minutes} minutes and {seconds} seconds")


Overall Time: 0 minutes and 37 seconds


In [6]:
# Show the result
result_df.head()

,customer_id,f_name,l_name,product_name,product_type,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1492,Ashley,Reed,Kelly's Spy Thriller,Event,$21.83,2023-10-13 21:59:32,International Distribution,Williams Ltd,Pacoima,Distribution Income Revenue 366640,Revenue
1,455,John,Romero,Brittany's Crazy Comedy,Subscription,$5.58,2023-11-09 05:12:50,Streaming Aggregators,Carter-Mcmahon,Richmond,Sell-Through Income Revenue 485857,Revenue
2,82,Robert,Barber,Lori's Date With Destiny,TV Series,$14.23,2023-04-15 08:16:54,Airports and Airlines,"Garcia, Keller and Collins",Chicago,Interest Income Revenue 306444,Revenue
3,389,Angela,Cohen,Charles's Patriotic Duty,Subscription,$11.34,2023-03-23 16:19:14,Educational Institutions,Savage and Sons,Charlotte,Streaming Cash Revenue 178957,Revenue
4,19,Alyssa,Miller,Michael's Wild Ride,Subscription,$15.18,2023-10-17 16:55:23,Virtual Reality (VR) Platforms,Schmidt-Mullins,Houston,Streaming Cash Revenue 811652,Revenue


In [7]:
dbcust = custom_excel_output(current_database="customers",svr_type='mysql')        

2003 (HY000): Can't connect to MySQL server on '127.0.0.1:3306' (111)
Database : customers, Connection Good: False


In [8]:

file_name = os.path.join(dbcust.get_this_dir(),"project_data","tempdir","customer_account_activity.xlsx")

result_df["amt_usd"] = result_df["amt_usd"].replace('[\$,]', '', regex=True).astype(float)

l_dfs = []
l_dfs.append(result_df)
dbcust.write_excel_from_dfs(list_of_dfs=l_dfs,file_name=file_name,add_subtotal_on_top=True)

href_tag = dbcust.get_embedded_href_tag_from_image_file(file_name)
dbcust.nukefile(file_name)
display(HTML(href_tag))
shuffled_df = result_df.sample(frac=1)
display(shuffled_df.head(5).style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])]))

,customer_id,f_name,l_name,product_name,product_type,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
8679,1148,Tiffany,Woodard,Tanya's Trip in the Wild,TV Season,16.110000,2023-11-20 19:33:22,Theatrical,Brown-Aguilar,El Cajon,Theatrical Income Revenue 138482,Revenue
6266,1599,Juan,Peters,Jeffrey's Monster Madness,Single View Movie,26.420000,2023-11-16 09:29:32,Home Video,"Robbins, Douglas and Morgan",Cedar Park,Theatrical Income Revenue 138482,Revenue
4821,457,Victoria,Baldwin,Andrew's Crazy Comedy,Pay TV,13.780000,2023-08-18 22:24:48,Music Publishing,"Gibson, Cox and Thomas",Richmond,Streaming Cash Revenue 555875,Revenue
6249,1713,Leah,Hill,Katie's Wild Ride,TV Season,14.330000,2023-06-06 00:08:55,Home Video,"Mullins, Olson and Fisher",West Lafayette,Streaming Cash Revenue 197554,Revenue
10494,598,Stephanie,Nelson,Alexander's Amazing Race,Event,22.550000,2023-03-23 06:19:11,Film Festivals,"Harris, Owens and Brown",Mesa,Distribution Income Revenue 245395,Revenue
